In [2]:
import spotipy
import time
from requests import ReadTimeout
import json 
import matplotlib.pyplot as plt
import numpy as np

In [3]:
data= json.load(open('metadata/ids.json'))
username = data["username"]
client_id = data["client-id"]
client_secret = data["client-secret"]
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

In [4]:
token = spotipy.util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)
# print(token)
sp= spotipy.Spotify(auth=token)

In [4]:


def get_ids():
    recently_played = sp.current_user_recently_played(limit=50)
    with open('metadata/recently_played.json', 'w') as outfile:
        json.dump(recently_played, outfile)

    before_dates=[]
    return ret_ids(recently_played)

def ret_ids(recently_played):
    track_ids=[]
    for track in recently_played['items']:
        # print(track["track"]["id"])
        track_ids.append(track["track"]["id"])
        # before_dates.append(track["track"]["cursors"]["before"])
    return track_ids

In [22]:
def get_features(track_ids):
    try:
        features = sp.audio_features(track_ids)
    except ReadTimeout as e:
        print("sleeping..")
        time.sleep(5)
        features = sp.audio_features(track_ids)
    features_data=json.loads(json.dumps(features))
    with open('metadata/features.json', 'w') as outfile:
        json.dump(features_data, outfile)
    # print(features_data)
    return features_data

def unix_time():
    # print(int(str(time.time()).split('.')[0]))
    return int(str(time.time()).split('.')[0])

In [23]:
def plot_features(features):
    categories=["danceability", "energy", "loudness", "key","mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]
    features=json.load(features)
    for category in categories:
        plt.figure(figsize=(8, 8))
        y=[feature[category] for feature in features]
        # y=
        x=[feature["id"] for feature in features]
        plt.plot(x,y)
        plt.savefig("plots/"+str(category)+".png")

In [25]:
def audio_analysis():
    with open("metadata/recently_played.json", "r") as f:
        data=json.load(f)
        ids=ret_ids(data)
        # print(ids)
        analysis_list=[]
        for id in ids:
            aa=sp.audio_analysis(id)
            analysis_list.append(aa)
        # print(analysis_list)

In [26]:
def average_features(features):
    categories=["danceability", "energy", "loudness", "key","mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]
    # features=json.load(features)
    averages={}
    for category in categories:
        y=[feature[category] for feature in features]
        averages[category]=sum(y)/len(y)
    return averages

In [27]:
def create_batch(features):
    batches=[]
    for i in range(0,len(features),50):
        batches.append(features[i:i+50])
    return batches

In [104]:
def filter_with_features(avgs):
    with open("metadata/top_tracks_per_similar_artist.json", "r") as f:
        tracks=json.load(f)
    batches=create_batch(tracks)
    tracks_with_features=[]
    for batch in batches:
        tracks_with_features+=get_features([track[1]for track in batch])
    categories=["danceability", "energy", "loudness", "key","mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]
    filtered_tracks=[]
    # count=[]
    for track in tracks_with_features:
        # count[track.index()]=0
        count=0
        # print(track.keys())
        if track is not None:
            for category in categories:
                # print(str(track[category])+"                 cc         " +str(np.arange(0.9*avgs[category],1.1*avgs[category],0.001)))
                if track[category]  >=0.9*avgs[category]and track[category]<=1.1*avgs[category]:
                    count+=1
        # break
        if count>3:
            try:
                filtered_tracks.append(track)
            except Exception as e:
                print(e )
                pass
    return filtered_tracks

In [105]:
# def main():
#     ids=get_ids()
#     features=get_features(ids)
#     with open("metadata/top_tracks_per_similar_artist.json", "r") as f:
#         tracks=json.load(f)
#         print(tracks.__len__())
#     filtered_tracks=filter_with_features(average_features(features))
#     print(filtered_tracks.__len__())
#     print(filtered_tracks)
#     # audio_analysis()
# main()
# test()

In [106]:
ids=get_ids()
features=get_features(ids)
with open("metadata/top_tracks_per_similar_artist.json", "r") as f:
    tracks=json.load(f)
    print(tracks.__len__())
filtered_tracks=filter_with_features(average_features(features))
print(filtered_tracks.__len__())
print(filtered_tracks)

7407
204
[{'danceability': 0.637, 'energy': 0.512, 'key': 2, 'loudness': -10.027, 'mode': 1, 'speechiness': 0.0978, 'acousticness': 0.59, 'instrumentalness': 0.000464, 'liveness': 0.0712, 'valence': 0.33, 'tempo': 109.492, 'type': 'audio_features', 'id': '1NlWBpNYAnFT7gMGyC0Y1M', 'uri': 'spotify:track:1NlWBpNYAnFT7gMGyC0Y1M', 'track_href': 'https://api.spotify.com/v1/tracks/1NlWBpNYAnFT7gMGyC0Y1M', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1NlWBpNYAnFT7gMGyC0Y1M', 'duration_ms': 179027, 'time_signature': 5}, {'danceability': 0.672, 'energy': 0.529, 'key': 7, 'loudness': -7.945, 'mode': 1, 'speechiness': 0.0597, 'acousticness': 0.274, 'instrumentalness': 0.0086, 'liveness': 0.171, 'valence': 0.328, 'tempo': 95.329, 'type': 'audio_features', 'id': '2iEkQcLF4Mjm91hTS3XW86', 'uri': 'spotify:track:2iEkQcLF4Mjm91hTS3XW86', 'track_href': 'https://api.spotify.com/v1/tracks/2iEkQcLF4Mjm91hTS3XW86', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2iEkQcLF4Mjm91hTS3XW8

In [5]:
print(sp.recommendation_genre_seeds())

{'genres': ['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep'